In [ ]:
import matplotlib, re, os, glob, datetime, difflib, random, time, math, json, wget
from datetime import date
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
import subprocess as sp
from bs4 import BeautifulSoup
from collections import defaultdict
sns.set('notebook')
%matplotlib inline 
# hide warnings
import warnings
warnings.filterwarnings('ignore')
from Bio import SeqIO, SeqUtils, SearchIO

In [ ]:
def cmdir(path):
    if not os.path.isdir(path):
        os.mkdir(path)

def scaffold(gene):
    if gene != "None":
        try: return re.search("(.+?)_[0-9]+$", gene).group(1)
        except: print(gene) 
            
def datestamp():
    return date.today().strftime("%Y-%m-%d")

In [ ]:
# define color palette
env2color = {
    "freshwater": "#5B7DAD",
    "sediment": "#312C29",
    "marine": "#10b5a7",
    "soil": "#7a5d1f",
    "engineered": "#B4B5B4",
    "animal-associated": "#A84726",
    "hypersaline": "#d6960b",
    "plant-associated": "#647d37"
}

In [ ]:
sns.palplot(env2color.values())

In [ ]:
rootdir = "TO_FILL_IN"
cmdir(rootdir)

# process metadata

In [ ]:
# assigning environment/study
current = sorted(glob.glob(rootdir + "metadata/filtered_genome_metadata_curated*"))[-1]
mc = pd.read_csv(current, sep=",")

In [ ]:
curated_public = pd.read_csv(rootdir + "metadata/crossenv_reads_public_2020_10_15_curated.tsv", sep="\t").fillna("None")
curated_public.head()

In [ ]:
curated_ggkbase = pd.read_csv(rootdir + "/metadata/crossenv_reads_ggkbase_2020_10_15_curated.tsv", sep="\t").fillna("None")
curated_ggkbase.head()

In [ ]:
curatedm = pd.concat([curated_public, curated_ggkbase]).fillna("None")
curatedm = curatedm.drop("env_broad", axis=1)
curatedm.head()

In [ ]:
# get run information
# NB VERY LONG RUNTIME
rundata = defaultdict(list)
leftovers = {}
count=0

for key, row in curatedm.iterrows():
    
    # those with SRA data
    if row["accession_curated"]!= "None":
        
        tokens = row["accession_curated"].replace(" ", "").split(",")
        sra_ids = []
        
        for token in tokens:
            
            prefix = re.search("([A-Z]+)[0-9]+", token).group(1)
            
            # first collect SRA ids
            if prefix in ["ERS", "SRS", "PRJEB", "SAMN", "PRJNA"]:
                
                # retreive sra ID
                try:
                    handle = Entrez.esearch(db='sra', term=token, RetMax=100)
                    result = Entrez.read(handle)
                    sra_ids += result['IdList']
                    handle.close()
                except:
                    print("Could not retrieve %s" %(token))
            
            else: # direct to SRA search
                sra_ids.append(token)
     
        for sra_id in sra_ids:
            
            try:
                
                if sra_id not in leftovers.keys():
                    
                    handle = Entrez.efetch(db="sra", id=sra_id, rettype="runinfo", retmode="xml")
                    soup = BeautifulSoup(handle, "lxml")
                    leftovers[sra_id] = soup
                
                else: 
                    soup = leftovers[sra_id]

                for run in soup.findAll("row"):

                    rundata["newname"].append(row["newname"])
                    rundata["host"].append(row["host"])
                    rundata["run_id"].append(run.find("run").text)
                    rundata["date"].append(run.find("releasedate").text)
                    rundata["spots"].append(run.find("spots").text)
                    rundata["bases"].append(run.find("bases").text)
                    rundata["library_type"].append(run.find("librarysource").text)
                    rundata["library_method"].append(run.find("librarystrategy").text)
                    rundata["library_layout"].append(run.find("librarylayout").text)
                    rundata["instrument"].append(run.find("model").text)
                    rundata["read_path"].append(row["read_path"])
                    rundata["assembly_path"].append(row["assembly_path"])

                handle.close()
            
            except:
                print("Could not retrieve %s" %(sra_id))

    # those without SRA data but valid ggk reads
    elif "/groups/" in row["read_path"]:
        
        read_sets = row["read_path"].replace(" ", "").split(",")
        
        for read_set in read_sets:
            
            rundata["newname"].append(row["newname"])
            rundata["host"].append(row["host"])
            rundata["run_id"].append(row["bioproject"])  
            rundata["date"].append("None")
            rundata["spots"].append("None")
            rundata["bases"].append("None")
            rundata["library_type"].append("METAGENOMIC")
            rundata["library_method"].append("WGS")
            rundata["library_layout"].append("PAIRED")
            rundata["instrument"].append("None")
            rundata["read_path"].append(read_set)
            rundata["assembly_path"].append(row["assembly_path"])   
    
    count+=1
    print('%d of %d genomes retrieved.\r'%(count, len(curatedm)), end="")

In [ ]:
#rundf = pd.DataFrame(rundata)
# filter it down
rdf = rundf[(~rundf["library_type"].isin(["METATRANSCRIPTOMIC", "GENOMIC SINGLE CELL"])) 
    & (~rundf["library_method"].isin(["AMPLICON", "RNA-Seq", "WGA"]))
    #& (~rundf["newname"].isin(to_remove))
    & (rundf["instrument"]!="MinION") & (rundf["library_layout"]!="SINGLE")]
# recast
rdf["spots"] = rdf["spots"].apply(lambda x: int(x) if x!="None" else "None")

In [ ]:
# write out fill rdf 
rdf.to_csv(rootdir + "/metadata/rdf.tsv", sep="\t", index=False)

### read processing + mapping

In [ ]:
cmdir(rootdir + "scripts")
cmdir(rootdir + "mapping")

In [ ]:
# read in ref genome paths
ref_genomes={line.split("\t")[0]: rootdir + "reference_genomes/animal/" + line.split("\t")[1].strip() 
    for line in open(rootdir + "reference_genomes/animal/reference_mappings.tsv").readlines()}

In [ ]:
#plot read # distribution
sns.distplot(rdf.query("spots != 'None'")["spots"], bins=150)
# arbitrary cutoff
plt.axvline(1E8)
plt.show()

In [ ]:
def compute_perc(value):
    
    goal = round(np.mean(rdf.query("spots != 'None'")["spots"]))
    return round(goal/float(value),2)

In [ ]:
# set up public read mapping + processing
callsets = []

for runid in rdf["run_id"].unique():
    
    temp = []
    basename = rootdir + "mapping/" + runid
    subtable = rdf[rdf["run_id"]==runid]
    readname = basename
    # define read paths
    forward = basename + ".1.fastq.gz"
    reverse = basename + ".2.fastq.gz"
    
    if ("SRR" in runid) or ("ERR" in runid):
        
        # download run
        temp.append("echo Starting download of %s... >> %s" %(runid, basename + ".log"))
        temp.append("parallel-fastq-dump -s " + runid + " -t 48 -O " +\
            rootdir + "mapping --split-files --gzip 2>> " + basename + ".log")
        # filter if necessary
        if subtable["spots"].to_list()[0] !="None":
            if subtable["spots"].to_list()[0] > 1e8:
                perc = compute_perc(subtable["spots"].to_list()[0])
                for side in ["1", "2"]:
                    temp.append("seqtk sample -s 7 " + basename + "_" + \
                        side + ".fastq.gz " + str(perc) + " > " + basename + "_" + side + ".SUB.fastq")
                    temp.append("pigz -p 48 " + basename + "_" + side + ".SUB.fastq")
                    temp.append("mv " + basename + "_" + side + ".SUB.fastq.gz " + basename + "_" + side + ".fastq.gz")
        # change names
        temp.append("mv " + basename + "_1.fastq.gz " + basename + ".1.fastq.gz")
        temp.append("mv " + basename + "_2.fastq.gz " + basename + ".2.fastq.gz")
    
    elif "groups" in subtable["read_path"].to_list()[0]:
        
        read_path = subtable["read_path"].to_list()[0]
        if "gz" not in read_path:
            read_path += ".gz"
        temp.append("cp " + read_path + " " + rootdir + "mapping/" + runid + ".1.fastq.gz")
        temp.append("cp " + read_path.replace(".1.fa", ".2.fa") + " " + \
            rootdir + "mapping/" + runid + ".2.fastq.gz")
    
    else: continue
    
    # filter reads for host
    host = subtable["host"].tolist()[0]
    if host in ref_genomes:
        temp.append("echo Filtering out %s reads... >> %s" %(host, basename + ".log"))
        temp.append("bbduk.sh threads=48 qhdist=1 in1=%s in2=%s out1=%s out2=%s ref=%s" %(
            forward, reverse, forward.replace(runid, runid + "_decontam"), reverse.replace(runid, runid + "_decontam"), ref_genomes[host]))
        readname = basename + "_decontam"
    
    # quality filter reads
    temp.append("echo Quality filtering reads... >> %s" %(basename + ".log"))
    temp.append("process_reads_bbmap.rb --basename " + readname + " -c -z -p 48")
    
    # gather genomes
    genomes = list(subtable["newname"])
    catfile = rootdir + "mapping/genomes_" + runid + ".fna"
    temp.append("echo Combining %s into %s... >> %s" %(",".join(genomes), os.path.basename(catfile), basename + ".log"))
    temp.append("cat %s > %s" %(" ".join([rootdir + "genome/" + genome + ".fna" for genome in genomes]), catfile))
    
    #build index, map
    temp.append("echo Mapping read files against %s... >> %s" %(os.path.basename(catfile), basename + ".log"))
    temp.append("bowtie2-build " + catfile + " " + catfile.replace(".fna", ""))
    temp.append("bowtie2 -p 48 -x " + catfile.replace(".fna", "") + " -1 " + \
        readname + "_trim_clean.PE.1.fastq.gz -2 " + readname + "_trim_clean.PE.2.fastq.gz 2>> " + basename + ".log" + \
        " | shrinksam | samtools view -S -b > " + basename + ".bam")
    temp.append("samtools sort --threads 48 " + basename + ".bam > " + basename + ".sorted.bam")
    temp.append("samtools index -@ 48 " + basename + ".sorted.bam")

    # delete read and build files
    temp.append("rm " + readname + ".[12].fastq* " + readname + "_trim.[12].fastq.* " + \
        readname + "*SR.fastq* " + rootdir + "mapping/genomes* " + readname + "_trim_clean.[12].fastq.*")
    temp.append("pigz -p 48 " + readname + "_trim_clean.PE.fa")
    temp.append("Success for %s... >> %s" %(runid, basename + ".log"))

    # if not already run, store
    try:
        size = os.stat(basename + ".bam").st_size
        if size == 0:
            callsets.append(temp)
    except FileNotFoundError:
        callsets.append(temp)

In [ ]:
# write to multiple wrappers
n = math.ceil(len(callsets)/1)
for i in range(0, len(callsets),n):
    with open(rootdir + "mapping/prc" + \
        str(int(i/n)+1) + ".sh", "w") as wrapper:
        for callset in callsets[i:i + n]:
            for call in callset:
                wrapper.write(call + "\n")

for item in $(ls | grep prc); do sbatch -J $item --wrap "$(pwd)/$item"; done

In [ ]:
# check for fail + rerun
count=0

for bam in glob.glob(rootdir + "mapping/*[0-9].bam"):
    # check file size
    if os.stat(bam).st_size == 0:
        sra = os.path.basename(bam).replace(".bam","")
        count+=1
        
print(count)

### analyze results

In [ ]:
# get non zero sorted bams
sorted_bams = [item for item in glob.glob(rootdir + "/mapping/*sorted.bam") if os.stat(item).st_size != 0]

In [ ]:
coverm = "coverm genome --genome-fasta-directory " + rootdir + "/genomes/ -x fna --min-read-percent-identity 0.99 " + \
    "--min-covered-fraction 0 --no-zeros --output-format sparse -b " + " ".join(sorted_bams) + " -m" + \
    " count mean covered_fraction length rpkm > " + rootdir + "/coverage_table.tsv"

out = open(rootdir + "/scripts/coverm.sh", "w")
out.write(coverm)
out.close()

Run manually in shell from `/scripts/` dir.

In [ ]:
covtable = pd.read_csv(rootdir + "coverage_table.tsv", sep="\t")
rdf = pd.read_csv(rootdir + "metadata/rdf.tsv", sep="\t")
covtable = covtable[covtable["Read Count"]>0].sort_values("Genome")
covtable["run_id"] = covtable["Sample"].apply(lambda x: x.split(".")[0])
bin2tax = {row["newname"]: row["taxcat"] for key, row in mc.iterrows()}
covtable["taxcat"] = covtable["Genome"].map(bin2tax)
# filter out bad run ids
valid = rdf["run_id"].unique()
covtable = covtable[covtable["run_id"].isin(valid)]
covtable.head()

In [ ]:
# read in previously computed read counts
read_counts = pd.concat(pd.read_csv(item, sep="\t") for item in glob.glob(rootdir + "/metadata/read_counts*.tsv"))
read_counts.columns = ["sample", "read_total"]
read_counts["run_id"] = read_counts["sample"].apply(lambda x: x.split("_trim_clean.PE.fa.gz")[0].replace("_decontam", ""))

# write out those not already computed
with open(rootdir + "metadata/read_files.txt", "w") as out:
    for key, row in covtable.drop_duplicates("Sample").iterrows():
        sample = row["Sample"].replace(".sorted","")
        # if not already computed
        if sample not in read_counts["run_id"].to_list():
            out.write(glob.glob(rootdir + "mapping/" + sample + "_*PE.fa.gz")[0] + "\n")

Then run `/scripts/computeReadCounts.py`.

In [ ]:
# merge in that + env data
meta = mc[["newname", "env_broad", "env_narrow"]].merge(curatedm[["newname", "host"]], how="left")
covmerge = covtable.merge(meta, how="left", left_on="Genome", right_on="newname")
covmerge = covmerge.merge(read_counts[["run_id", "read_total"]])
covmerge = covmerge[["run_id","env_broad", "env_narrow", "host", "Genome", "taxcat", "Read Count", "Covered Fraction", "read_total", "RPKM"]]
# compute relative abundance
covmerge["perc_reads"] = covmerge.apply(lambda x: (x["Read Count"]/float(x["read_total"]))*100 if x["read_total"] != "None" else 0, axis=1)
covmerge["log_perc_reads"] = covmerge["perc_reads"].apply(lambda x: math.log10(x) if x != 0 else 0)
covmerge.head()

In [ ]:
# select max per genome
covsub = covmerge.sort_values("log_perc_reads", ascending=False).drop_duplicates("Genome")
# implement cov frac min
covsub = covsub[covsub["Covered Fraction"]>=0.1]
covsub.head()

In [ ]:
# across env_broad
sns.set_style("ticks")
plt.figure(figsize=(3,7))
sns.boxplot("perc_reads","env_broad", hue="taxcat", color="lightgrey", linewidth=0.5, data=covsub, fliersize=0)
sns.stripplot("perc_reads", "env_broad", hue="taxcat", data=covsub, dodge=True, size=3)
plt.xticks(rotation=45, horizontalalignment="right")
plt.xscale("log")
plt.grid('on', which='major', axis='x' )
plt.xlabel("max percent reads")
plt.ylabel("")
plt.legend(bbox_to_anchor=(1, 1), loc='upper left')
plt.savefig(rootdir + "figures/env_broad.svg", format="svg", bbox_inches="tight")
plt.show()

In [ ]:
# among animal assc
sns.set_style("ticks")
plt.figure(figsize=(3,4))
subset = covsub[covsub["env_narrow"].isin(["animal gut", "animal oral", "human gut", "human oral"])].sort_values("env_narrow")
sns.boxplot("perc_reads","env_narrow", hue="taxcat", color="lightgrey", linewidth=0.5, data=subset, fliersize=0)
sns.stripplot("perc_reads", "env_narrow", hue="taxcat", data=subset, dodge=True, size=3)
plt.xticks(rotation=45, horizontalalignment="right")
plt.xscale("log")
plt.grid('on', which='major', axis='x' )
plt.xlabel("max percent reads")
plt.ylabel("")
plt.legend(bbox_to_anchor=(1, 1), loc='upper left')
plt.savefig(rootdir + "figures/env_narrow.svg", format="svg", bbox_inches="tight")
plt.show()

### supp table 2

In [ ]:
s2 = covsub
s2["accession"] = s2["run_id"].apply(lambda x: x if "RR" in x else "None")
s2 = s2[["Genome", "taxcat", "env_broad", "env_narrow", "run_id", "accession", "Read Count", "read_total", "perc_reads", "log_perc_reads", "Covered Fraction"]]
s2.columns = ["new_bin_name", "lineage", "habitat_broad", "habitat_narrow", "run_id", "accession", "mapped_read_count", "sample_read_total", "mapped_read_percent", "log_mapped_read_percent", "fraction_genome_covered"]
s2.sort_values(["lineage", "habitat_broad", "habitat_narrow", "run_id"]).to_csv(rootdir + "metadata/supp_table_2.csv", index=False)

# read assembly

In [ ]:
cmdir(rootdir + "assembly")

In [ ]:
def wrapperize(calls, parts, out, dtype):
    
    n = math.ceil(len(calls)/parts)
    for i in range(0, len(calls),n):
        with open(out + \
            str(int(i/n)+1) + ".sh", "w") as wrapper:
            for call in calls[i:i + n]:
                if dtype == "str":
                    wrapper.write(call + "\n")
                elif dtype == "list":
                    for subcall in call:
                        wrapper.write(subcall + "\n")
    print("chmod +x " + out + "*")
    print('for item in $(ls ' + out + '*); do sbatch --wrap "$item"; done')

### assemble reads

In [ ]:
runsub = covsub[["run_id", "Genome"]].merge(rdf[["run_id", "assembly_path"]])
# add back in read locations once computed
runsub["read_path"] = runsub["run_id"].apply(lambda x: glob.glob(rootdir + \
    "/mapping/" + x + "_*PE.fa.gz")[0] if glob.glob(rootdir + "/mapping/" + x + "_*PE.fa.gz") != [] else "None")
runsub.head()

In [ ]:
megas = []
eligible = []

for key, row in runsub[["run_id", "read_path", "assembly_path"]].drop_duplicates().iterrows():
    
    if (row["read_path"] != "None") & (row["assembly_path"]=="None") :
        
        base = os.path.basename(row["run_id"])
        eligible.append(base)
        
        # if not already successfully assembled
        if not os.path.isfile(rootdir + "/assembly/" + base + "/done"):
            megas.append("megahit --verbose --min-contig-len 1000 -t 48 --12 " + \
                row["read_path"]+ " -o " + rootdir + "/assembly/" + base + "/ --out-prefix " + base)

In [ ]:
# any that need to be deleted?
previously_computed = [path.split("/")[-2] for path in glob.glob(rootdir + "assembly/*/")]
to_delete = list(set(previously_computed) - set(eligible))

print(to_delete)

for item in to_delete:
    call = "rm -r " + rootdir + "assembly/" + item
    #sp.call(call, shell=True)

In [ ]:
wrapperize(megas, 25, rootdir + "assembly/assem")

### format assemblies

In [ ]:
# rename SRA assembly scafs

for assembly_dir in glob.glob(rootdir + "assembly/*/"):
    
    name = assembly_dir.split("/")[-2]
    # only if assembly done
    if os.path.isfile(assembly_dir + "done"):
        contigs = glob.glob(assembly_dir + "*contigs.fa*")[0]
        renamed = contigs.replace("contigs", "renamed")
        
        # if not already renamed
        if not os.path.isfile(renamed):
            
            count=1
            with open(renamed,"w") as new_assembly:
                for record in SeqIO.parse(open(contigs), "fasta"):
                    new_assembly.write(">" + name + "_scaffold_" + \
                        str(count) + "\n" + str(record.seq) + "\n")
                    count+=1

In [ ]:
# rename GGK assembly scafs

for key, row in runsub[runsub["assembly_path"]!="None"].drop_duplicates(["run_id", "assembly_path"]).iterrows():
    
    # if not already done
    if not os.path.isdir(rootdir + "assembly/" + row["run_id"]):
        
        # find actual assembly
        clean_path = "/".join(row["assembly_path"].split("/")[:-1])
        ass_path = glob.glob(clean_path + "/*min[0-9]000.fa")
        
        if len(ass_path) == 1:

            count=1
            cmdir(rootdir + "assembly/" + row["run_id"])
            with open(rootdir + "assembly/" + row["run_id"] + "/" + row["run_id"] + ".renamed.fa", "w") as newass:
                for record in SeqIO.parse(open(ass_path[0]), "fasta"):
                    newass.write(">" + row["run_id"] + "_scaffold_" + \
                        str(count) + "\n" + str(record.seq) + "\n")
                    count+=1

In [ ]:
# add back assembly paths for ALL run_ids
runsub["curated_assembly_path"] = runsub["run_id"].apply(lambda x: glob.glob(rootdir + \
    "/assembly/*/" + x + ".renamed.fa")[0] if glob.glob(rootdir + "/assembly/*/" + x + ".renamed.fa") != [] else "None")
runsub.head()

In [ ]:
# which need to be re-assembled later?
runsub[runsub["curated_assembly_path"]=="None"]["run_id"].unique()

In [ ]:
# output table for use elsewhere
temp = runsub[["Genome", "run_id", "read_path", "curated_assembly_path"]].drop_duplicates()
temp.columns = ["genome","run_id", "read_path", "assembly_path"]
temp.to_csv(rootdir + "metadata/curated_run_info.tsv", sep="\t", index=False)

# marker gene analysis

In [ ]:
cmdir(rootdir + "graftm")

### run graftm

In [ ]:
# define gpkgs
s3 = rootdir + "other/S3.gpkg"

In [ ]:
grafts = []

for assembly in glob.glob(rootdir + "/assembly/*/*.renamed.fa"):
    
    runid = os.path.basename(assembly).split(".")[0]
    # if not already computed
    if not os.path.isdir(rootdir + "graftm/" + runid):
        call = "graftM graft --threads 48 --forward " + \
            assembly + " --input_sequence_type nucleotide --graftm_package " + s3 + \
            " --output_directory " + rootdir + "graftm/" + runid
        grafts.append(call)

In [ ]:
wrapperize(grafts, 4, rootdir + "graftm/grm")

### perform clustering

In [ ]:
## from RpxSuite - https://github.com/alexcritschristoph/
    ##RPxSuite/blob/master/rpXsuite/RPxSuite.py

def parse_usearch_clustering(loc):

    dtypes = {0:'category', 1:'category', 2:np.int32, 8:'object'}
    ucols = [0,1,2,8]
    Rdb = pd.read_csv(loc, header=None, usecols=ucols,\
            dtype=dtypes, sep='\t')
    table = defaultdict(list)

    # Find the centroids
    sdb  = Rdb[Rdb[0] == 'S']
    shdb = Rdb[Rdb[0].isin(['H', 'S'])]
    for centroid, cdb in sdb.groupby(1):
        cent = cdb[8].tolist()[0].split()[0]
        db = shdb[shdb[1] == centroid]

        for seq in db[8].tolist():
            table['cluster'].append(int(centroid))
            table['members'].append(len(db))
            table['sequence'].append(seq.split()[0])
            table['centroid'].append(cent)

    return pd.DataFrame(table)

In [ ]:
cresults = {}

for graftdir in glob.glob(rootdir + "graftm/*/*/"):
        
    name = graftdir.split("/")[-2].replace(".renamed", "")
    # find files
    taxinfo = glob.glob(graftdir + "*renamed_read_tax.tsv")[0]
    
    # only proceed if results
    if os.stat(taxinfo).st_size != 0:
        
        full_fasta = glob.glob(graftdir + "*renamed_orf.fa")[0]
        # perform clustering
        call = "usearch -cluster_fast " + full_fasta + " -sort length -id 0.99 " + \
            "-centroids " + full_fasta.replace("renamed_orf", "centroids") + \
            " -uc " + taxinfo.replace("renamed_read_tax.tsv", "clusters")
        sp.call(call, shell=True)

        # process clustering
        cluster_results = parse_usearch_clustering(taxinfo.replace("renamed_read_tax.tsv", "clusters"))
        # merge tax calls
        cluster_results = cluster_results.merge(pd.read_csv(taxinfo, sep="\t", names=["sequence", "taxstring"]))
        # save out
        cresults[name] = cluster_results[["cluster", "members", "centroid", "taxstring"]].drop_duplicates()
    
    else: print("%s had no marker gene hits!" %(name))
    

### analyze

In [ ]:
cluster_counts = defaultdict(list)

for key, results in cresults.items():
    
    cluster_counts["run_id"].append(key)
    cluster_counts["uniq_actino"].append(len(results[results["taxstring"].str.contains("p__Actinobacteriota")]))
    cluster_counts["uniq_sacchari"].append(len(results[results["taxstring"].str.contains("c__Saccharimonadia")]))

countsdf = pd.DataFrame(cluster_counts)
countsdf.head()

Run `sum-bp  assembly/*/*renamed.fa > metadata/sumbp_results.txt` in the terminal to get assembly totals.

In [ ]:
# add in sum-bp results
sums = {line.split(" ")[0].split(".")[0]: int(line.split(" ")[1].strip().replace(",", "")) 
    for line in open(rootdir + "metadata/sumbp_results.txt").readlines()}

countsdf["sumbp"] = countsdf["run_id"].map(sums)
countsdf["megabases"] = countsdf["sumbp"].apply(lambda x: x/float(1e6))

# compute per Mb counts
for tax in ["actino", "sacchari"]:
    countsdf[tax + "_permb"] = countsdf.apply(lambda x: x["uniq_" + tax]/float(x["megabases"]), axis=1)

# finally merge in environments
countsdf = countsdf.merge(covmerge[["run_id", "env_broad", "env_narrow"]], how="left").drop_duplicates()

In [ ]:
# how many with no saccharis assembled?
countsdf[(countsdf["sacchari_permb"]==0) | (countsdf["actino_permb"]==0)]["env_narrow"].value_counts()

In [ ]:
# plot
nozed = countsdf[(countsdf["actino_permb"]>0) & (countsdf["sacchari_permb"]>0)]
x = np.linspace(0, 10, 1000)

plt.figure(figsize=(5,5))
sns.scatterplot("actino_permb", "sacchari_permb", data=nozed, hue="env_broad", palette=env2color, legend=None)
plt.xscale("log")
plt.yscale("log")
plt.xlim(10e-4,0)
plt.ylim(10e-4, 0)
plt.xlabel("normalized species richness (Actinobacteria)")
plt.ylabel("normalized species richness (Saccharibacteria)")
plt.show()

In [ ]:
nozed["ratio"] = nozed.apply(lambda x: x["uniq_sacchari"]/x["uniq_actino"], axis=1)
order = nozed.groupby("env_broad", as_index=False).aggregate({"ratio":"mean"}).sort_values("ratio", ascending=False)["env_broad"].to_list()

In [ ]:
plt.figure(figsize=(6,5))
sns.boxplot("ratio", "env_broad",color="white", order=order, linewidth=0.5, data=nozed, width=0.6, fliersize=0)
sns.stripplot("ratio", "env_broad", hue="env_broad", order=order, palette=env2color, linewidth=0.1, data=nozed, size=4)
plt.xticks(rotation=40, horizontalalignment="right")
plt.ylabel("")
plt.xlabel("richness ratio")
plt.axvline(1, ls="--", color="grey")
plt.legend(bbox_to_anchor=(1, 1), loc='upper left')
plt.xscale("log")
plt.savefig(rootdir + "figures/ratio.svg", format="svg", bbox_inches="tight")
plt.show()

In [ ]:
countsdf["missing"] = countsdf.apply(lambda x: 1 if ((x["uniq_actino"]==0)) else 0, axis=1)
countsdf["present"] = countsdf.apply(lambda x: 1 if ((x["uniq_actino"]!=0)) else 0, axis=1)
countgb = countsdf.groupby("env_broad", as_index=False).aggregate({"missing":"sum", "present": "sum"})

for cat in ["missing", "present"]:
    countgb[cat + "_perc"] = countgb.apply(lambda x: int(x[cat])/float(int(x["missing"]) + int(x["present"]))*100, axis=1)

countgb.index = countgb["env_broad"]
countgb = countgb.drop(["env_broad", "missing", "present"], axis=1)[["present_perc", "missing_perc"]]
order.reverse()
countgb.loc[order,:].plot.barh(color=["darkgrey", "white"], stacked=True, edgecolor="grey", width=0.6, linewidth=0.5, figsize=(1,5))
plt.legend(bbox_to_anchor=(1, 1), loc='upper left')
plt.ylabel(" ")
plt.xlabel("%")
plt.tick_params(left=False)
sns.despine(left=True)
plt.savefig(rootdir + "figures/bar.svg", format="svg", bbox_inches="tight")

cmdir(rootdir + "trees")### across samples

In [ ]:
cmdir(rootdir + "trees")
all_results = pd.concat(cresults.values())

In [ ]:
# concat all markers to use as db
with open(rootdir + "trees/all_rps3.faa", "w") as catfile:
    for graftdir in glob.glob(rootdir + "graftm/*/*/"):
        if os.stat(glob.glob(graftdir + "*renamed_read_tax.tsv")[0]).st_size != 0:
            for record in SeqIO.parse(open(glob.glob(graftdir + "*renamed_orf.fa")[0]), "fasta"):
                # only copy over if already a within-sample centroid
                if record.description in all_results["centroid"].to_list():
                    catfile.write(">%s\n%s\n" %(record.description.split(" ")[0], str(record.seq)))  

In [ ]:
# cluster across samples
call = "usearch -cluster_fast " + rootdir + "/trees/all_rps3.faa -sort length -id 0.99 " + \
    "-centroids " + rootdir + "/trees/all_rps3.centroids" + \
    " -uc " + rootdir + "/trees/all_rps3.clusters"
sp.call(call, shell=True)

In [ ]:
# process clustering
cluster_results = parse_usearch_clustering(rootdir + "/trees/all_rps3.clusters")
# merge back taxstring
cluster_results = cluster_results.merge(all_results[["centroid", "taxstring"]], on="centroid", how="left").fillna("None")
cluster_results.head()

### get experimental tm7-host pairs

In [ ]:
import skbio.io
from io import StringIO
cmdir(rootdir + "reference_genomes/cpr-actino")

In [ ]:
# read in compiled metadata
hostdf = pd.read_csv(rootdir + "/metadata/crossenv_sacchari_hosts.tsv", sep="\t")
hostdf.head()

In [ ]:
# get the genomes
sacc = [item for item in list(hostdf["sacchari_accession"].unique()) if item != "None"]
aacc = [item for item in list(hostdf["host_accession"].unique()) if item != "None"]
" OR ".join(sacc + aacc)

Grab from NCBI and move into dir, unzip.

In [ ]:
# join it up
hostdf["sfilename"] = hostdf["sacchari_accession"].apply(lambda x: [item for item in glob.glob(rootdir + "reference_genomes/cpr-actino/*") if x in item][0] if x!= "None" else "None")
hostdf["hfilename"] = hostdf["host_accession"].apply(lambda x: [item for item in glob.glob(rootdir + "reference_genomes/cpr-actino/*") if x in item][0] if x!= "None" else "None")

cat *fna > all_references.fna

In [ ]:
#extract s3
call = "graftM graft --threads 16 --forward " + \
        rootdir + "reference_genomes/cpr-actino/all_references.fna" + \
        " --input_sequence_type nucleotide --graftm_package " + s3 + \
        " --output_directory " + rootdir + "reference_genomes/cpr-actino/all_references"
print(call)

In [ ]:
with open(rootdir + "/scripts/blast_markers.sh", "w") as out:
    
    # make blast db
    makedb = "makeblastdb -in " + rootdir + "trees/all_rps3.centroids -dbtype prot"
    # perform blast
    blast = "blastp -db %s -query %s -out %s -evalue 1e-3 -max_target_seqs 10 -num_threads 16 -sorthits 3 -outfmt 6" %(rootdir + "trees/all_rps3.centroids", rootdir + "reference_genomes/cpr-actino/all_references/all_references/all_references_orf.fa", rootdir + "trees/reference.blast.results")
    out.write(makedb + "\n" + blast + "\n")

In [ ]:
# get ref seq lens
ref_lens = {record.description.split(" ")[0]:len(record.seq) 
    for record in SeqIO.parse(open(rootdir + "reference_genomes/cpr-actino/all_references/all_references/all_references_orf.fa"), "fasta")}

In [ ]:
bresults = skbio.io.read(rootdir + "/trees/reference.blast.results", format="blast+6", into=pd.DataFrame, default_columns=True)
# compute coverage
bresults["qlen"] = bresults["qseqid"].map(ref_lens)
bresults["qcov"] = bresults.apply(lambda x: (x["qend"]-x["qstart"])/x["qlen"], axis=1)
# choose best hits for each
bresults = bresults.sort_values(["bitscore", "qcov"], ascending=[False,False]).drop_duplicates("qseqid")
# make scaf2bin
bresults["scaffold"] = bresults["qseqid"].apply(lambda x: x.split("_")[0])
# scaffold 2 bin
refscaf2bin = {}
for file in glob.glob(rootdir + "reference_genomes/cpr-actino/GCA*fna"):
    for record in SeqIO.parse(open(file), "fasta"):
        refscaf2bin[record.description.split(" ")[0]] = file
bresults["filename"] = bresults["scaffold"].map(refscaf2bin)
bresults.head()

In [ ]:
# filter matches at 99% PID (to match clustering) and 95% query coverage
filename2marker = {}

for key, row in bresults.iterrows():
    
    if (row["pident"]>=99) and (row["qcov"]>=0.95):
        filename2marker[row["filename"]] = row["sseqid"]
    else: #maintain original name
        filename2marker[row["filename"]] = row["qseqid"]

In [ ]:
# these edges may or may not exist already in the dataframe,
# so in case they have not been oberved, reconstruct auxiliary df
aux = defaultdict(list)

for key, row in hostdf.iterrows():
    
    try:
        smarker = filename2marker[row["sfilename"]]
        hmarker = filename2marker[row["hfilename"]]
        aux["scentroid"].append(smarker)
        aux["acentroid"].append(hmarker)
        
    except:
        print("%s with %s is missing." %(row["sacchari_name"], row["host_name"]))

### supp table 3

In [ ]:
# get sequences
seq_dict = {record.description.split(" ")[0]: str(record.seq) for record in SeqIO.parse(open(rootdir + "trees/all_rps3.faa"), "fasta")}
ref_dict = {record.description.split(" ")[0]: str(record.seq) for record in SeqIO.parse(open(rootdir + "reference_genomes/cpr-actino/all_references/all_references/all_references_orf.fa"), "fasta")}

In [ ]:
bresults_sub = bresults[(bresults["pident"]>=99) & (bresults["qcov"]>=0.95)][["filename", "sseqid", "pident", "qcov"]]
bresults_sub["pident"] = bresults_sub["pident"].apply(lambda x: round(x, 5))
bresults_sub["qcov"] = bresults_sub["qcov"].apply(lambda x: round(x, 5))
bresults_sub.head()

In [ ]:
hostdf["saccharibacteria_marker"] = hostdf["sfilename"].map(filename2marker).fillna("None")
hostdf["host_marker"] = hostdf["hfilename"].map(filename2marker).fillna("None")
s3 = hostdf.merge(bresults_sub[["filename", "pident", "qcov"]], how="left", left_on="sfilename", right_on="filename")
s3 = s3.merge(bresults_sub[["filename", "pident", "qcov"]], how="left", left_on="hfilename", right_on="filename")
s3 = s3.drop(["sfilename", "hfilename", "notes","filename_x", "filename_y"], axis=1)
s3.columns = ["saccharibacteria_name", "saccharibacteria_accession", "host_name", "host_accession", "reference", "saccharibacteria_s3_marker", "host_s3_marker", "saccharibacteria_marker_pid", "saccharibacteria_marker_cov", "host_marker_pid", "host_marker_cov"]
# remove those with no S3 or no ref genome
s3 = s3[(s3["saccharibacteria_s3_marker"]!='None') & (s3["host_s3_marker"]!="None")]
# now add sequences
s3["saccharibacteria_s3_seq"] = s3["saccharibacteria_s3_marker"].apply(lambda x: seq_dict[x] if x in seq_dict.keys() else ref_dict[x])
s3["host_s3_seq"] = s3["host_s3_marker"].apply(lambda x: seq_dict[x] if x in seq_dict.keys() else ref_dict[x])
s3 = s3[['saccharibacteria_name', 'saccharibacteria_accession', 'host_name', 'host_accession', 'reference',
        'saccharibacteria_s3_marker', 'saccharibacteria_marker_pid', 'saccharibacteria_marker_cov', 'saccharibacteria_s3_seq',
        'host_s3_marker', 'host_marker_pid', 'host_marker_cov', "host_s3_seq"]]

In [ ]:
s3.fillna("None").to_csv(rootdir + "metadata/supp_table_3.csv", index=False)

### tree-building

In [ ]:
from ete3 import Tree

In [ ]:
def parse_hmm(result_table):
    
    temp = {}
    count = 0
    # parse each result file using searchio
    for result in SearchIO.parse(result_table, "hmmer3-tab"):
        for item in result.hits:
            temp[count] = {"gene": item.id, "score": item.bitscore, "eval": item.evalue}
            count += 1
            
    return(pd.DataFrame.from_dict(temp, orient="index"))

In [ ]:
# build unmatched ref seqs
unmatched = bresults[(bresults["pident"]<99) | (bresults["qcov"]<0.95)]
unmatcheds = unmatched[bresults["filename"].isin(hostdf["sfilename"])]["qseqid"].to_list()
unmatcheda = unmatched[bresults["filename"].isin(hostdf["hfilename"])]["qseqid"].to_list()

with open(rootdir + "trees/saccharimonadia_unmatched.faa", "w") as out:
    for record in SeqIO.parse(open(rootdir + "reference_genomes/cpr-actino/all_references/all_references/all_references_orf.fa"), "fasta"):
        if record.description.split(" ")[0] in unmatcheds:
            out.write(">" + record.description.split(" ")[0] + "\n" + str(record.seq) + "\n")
            
with open(rootdir + "trees/actinobacteriota_unmatched.faa", "w") as out:
    for record in SeqIO.parse(open(rootdir + "reference_genomes/cpr-actino/all_references/all_references/all_references_orf.fa"), "fasta"):
        if record.description.split(" ")[0] in unmatcheda:
            out.write(">" + record.description.split(" ")[0] + "\n" + str(record.seq) + "\n")

In [ ]:
# build bac175 outgroup
call = "hmmsearch --cut_nc --tblout " + rootdir + "trees/bac175.s3.results " + \
    rootdir + "other/TIGR01009.HMM " + rootdir + "reference_genomes/bac175/Bacteria175.cleaned.faa"
sp.call(call, shell=True)

with open(rootdir + "trees/bac175.s3.txt", "w") as out:
    for key, row in parse_hmm(rootdir + "trees/bac175.s3.results").iterrows():
        # remove actinos
        if row["gene"] not in []:
             out.write(row["gene"] + "\n")

call = "pullseq -n " + rootdir + "trees/bac175.s3.txt -i " + \
    rootdir + "reference_genomes/bac175/Bacteria175.cleaned.faa > " + rootdir + "trees/bac175.s3.faa"
sp.call(call, shell=True)

In [ ]:
wrapper = open(rootdir + "trees/wrapper.sh", "w")

for term in ["p__Actinobacteriota", "c__Saccharimonadia"]:
    
    sub = cluster_results[(cluster_results["taxstring"].str.contains(term))]
    names_file = rootdir + "trees/" + term.split("_")[2].lower() + "_rps3.txt"
    
    with open(names_file, "w") as outfile:
        for key, row in sub.iterrows():
            outfile.write(row["centroid"]+'\n')
    
    # create fasta file
    pull = "pullseq -n " + names_file + " -i " + \
        rootdir + "trees/all_rps3.faa > " + names_file.replace(".txt", ".faa")
    
    # add out group and unmatched refs
    cat = "cat " + rootdir + "trees/bac175.s3.faa " + names_file.replace("rps3.txt", "unmatched.faa") + \
        " " + names_file.replace(".txt", ".faa") + " > " + names_file.replace(".txt", ".cat.faa")
    
    # align + trim
    align = "mafft --thread 16 --retree 2 --reorder " + \
        names_file.replace(".txt", ".cat.faa") + " > " + names_file.replace(".txt", ".mafft")
    
    # trimal call
    trim = "trimal -in " + names_file.replace(".txt", ".mafft") + " -out " + \
        names_file.replace(".txt", ".trimal.mafft") + " -gt 0.1"
    
    # launch tree inference
    tree = "iqtree -s " + names_file.replace(".txt", ".trimal.mafft") + \
        " -m TEST -st AA -bb 1000 -nt 10 -pre " + names_file.replace(".txt", "")
    print(tree)
    wrapper.write(pull + "\n" + cat + "\n" + align + "\n" + trim + "\n")
    
wrapper.close()

In [ ]:
# write out itol
env_dict = {row["run_id"]:row["env_broad"] for key, row in covsub.fillna("None").iterrows()}

# for unmatched references
for item in unmatched["qseqid"].unique():
    env_dict[item] = "animal-associated"

In [ ]:
for group in ["saccharimonadia", "actinobacteriota"]:
    with open(rootdir + "/trees/" + group + "_rps3.itol.txt", "w") as itol:
        itol.write("TREE_COLORS\nSEPARATOR TAB\nDATA\n")
        for leaf in Tree(rootdir + "/trees/" + group + "_rps3.treefile"):
            if "BAC175" not in leaf.name:
                env = env_dict[leaf.name.split("_scaffold")[0]]
                if env != "None":
                    itol.write(leaf.name + "\trange\t" + env2color[env] + \
                        "\t" + env_dict[leaf.name.split("_scaffold")[0]] + "\n")

### co-occurrence plot

In [ ]:
# generate all comparisons
lines = defaultdict(list)

for key, table in cresults.items():
    
    sac = set(table[(table["taxstring"].str.contains("c__Saccharimonadia"))]["centroid"].to_list())
    act = set(table[(table["taxstring"].str.contains("p__Actinobacteriota"))]["centroid"].to_list())
    
    for s in sac:
        for a in act:
            lines["sample"].append(key)
            lines["scluster"].append(s)
            lines["acluster"].append(a)

linedf = pd.DataFrame(lines)

In [ ]:
# incorporate clustering of clustering
#sacchari
linedf = linedf.merge(cluster_results[["sequence", "centroid"]], how="left", left_on="scluster", right_on="sequence").drop("sequence", axis=1).rename(columns={"centroid": "scentroid"})
# actino
linedf = linedf.merge(cluster_results[["sequence", "centroid"]], how="left", left_on="acluster", right_on="sequence").drop("sequence", axis=1).rename(columns={"centroid": "acentroid"})
# then aggregate
lineagg = linedf.groupby(["scentroid", "acentroid"], as_index=False).count()[["scentroid", "acentroid", "sample"]]

In [ ]:
# TODO assign order based on phylogeny
ssort = {item:i for i, item in enumerate([line.replace(" ","_").strip() 
    for line in open(rootdir + "trees/saccharimonadia_rps3.order.txt").readlines()])}
asort = {item:i for i, item in enumerate([line.replace(" ","_").strip() 
    for line in open(rootdir + "trees/actinobacteriota_rps3.order.txt").readlines()])}
lineagg["snum"] = lineagg["scentroid"].map(ssort)
lineagg["anum"] = lineagg["acentroid"].map(asort)
# add back habitat info
lineagg["env_broad"] = lineagg["scentroid"].apply(lambda x: env_dict[x.split("_scaffold")[0]])

In [ ]:
def get_lcs(values):
    
    max_ranks = max([len(item.split(";")) for item in values])
    lowest_rank = ""
    
    for i in range(max_ranks):
        temp = []
        for item in values:
            splt = item.split(";")
            if len(splt) > i:
                temp.append(splt[i])
            else: temp.append("None")
        
        unique = set(temp)
        if (len(unique)==1) and (list(unique)[0]!="None"):
            lowest_rank = list(unique)[0]
    
    return lowest_rank

In [ ]:
centroid2tax = {row["centroid"]:row["taxstring"] 
    for key, row in all_results.iterrows()}
lineagg["stax"] = lineagg["scentroid"].map(centroid2tax)
lineagg["atax"] = lineagg["acentroid"].map(centroid2tax)
edges = {}

for scentroid in lineagg["scentroid"].unique():
    
    table = lineagg[lineagg["scentroid"]==scentroid]
    if table["env_broad"].iloc[0] == "animal-associated":
        lowest =  get_lcs(table["atax"].to_list())
        if ("g__" in lowest) or ("o__" in lowest):
            edges[scentroid] = "exclusive co-occurrence (%s)" %(lowest.split("__")[0].strip())

In [ ]:
for scluster in linedf["scluster"].unique():
    
    table = linedf[linedf["scluster"]==scluster]
    if len(table["acentroid"].unique()) == 1:
        edges[table["scentroid"].iloc[0]] = {table["acentroid"].iloc[0]: "exclusive co-occurrence (sg)"}

In [ ]:
def getType(row):
    
    if row["scentroid"] in edges.keys():
        
        if type(edges[row["scentroid"]]) == str:
            return edges[row["scentroid"]]
        
        elif row["acentroid"] == list(edges[row["scentroid"]].keys())[0]:
            return edges[row["scentroid"]][row["acentroid"]]
        
        else: return "co-occurrence"
    
    else: return "co-occurrence"

lineagg["type"] = lineagg.apply(lambda x: getType(x), axis=1)

In [ ]:
auxdf = pd.DataFrame(aux)
# merge in snum and anum
auxdf["snum"] = auxdf["scentroid"].map(ssort)
auxdf["anum"] = auxdf["acentroid"].map(asort)

In [ ]:
plt.figure(figsize=(18,3))
sspace = len(asort.keys())/(len(ssort.keys()))

for key, item in ssort.items():
    plt.plot(item*sspace, 0,'bo', color="lightgrey", ms=0.5)
for key, item in asort.items():
    plt.plot(item, 1,'bo', color="lightgrey", ms=0.5)
for key, row in lineagg.iterrows():
    if row["env_broad"] == "animal-associated":
        if row["type"] == "co-occurrence":
            plt.plot((row["snum"]*sspace, row["anum"]), (0,1), ls="-", alpha=0.1, linewidth=min(1*row["sample"], 8), color="lightgrey",zorder=0)
        elif "sg" in row["type"]:
            plt.plot((row["snum"]*sspace, row["anum"]), (0,1), ls="-", alpha=0.5, linewidth=min(2*row["sample"],8), color="lightgreen",zorder=9)
        else:
            plt.plot((row["snum"]*sspace, row["anum"]), (0,1), ls="-", alpha=0.5, linewidth=min(2*row["sample"], 8), color="pink",zorder=9)

# add in experimental ones
for key, row in auxdf.iterrows():
    plt.plot((row["snum"]*sspace, row["anum"]), (0,1), ls="-", alpha=0.5, linewidth=3, color="lightblue",zorder=9)#

plt.ylim([-0.1, 1.1])
plt.xlim([-10, max(asort.values())+10])
sns.despine(left=True, bottom=True)
plt.savefig(rootdir + "figures/occurrence.svg", format="svg")

### supp table 4

In [ ]:
sg = lineagg[lineagg["type"]=="exclusive co-occurrence (sg)"]
lines = []

for key, row in sg.iterrows():
    sub = linedf[(linedf["scentroid"]==row["scentroid"]) & (linedf["acentroid"]==row["acentroid"])]
    subsub = sub[sub["sample"].isin(countsdf[countsdf["uniq_actino"]==1]["run_id"].to_list())]
    lines.append(subsub)

sglines = pd.concat(lines)

In [ ]:
o = lineagg[lineagg["type"]=="exclusive co-occurrence (o)"]

lines = []

for key, row in o.iterrows():
    sub = linedf[(linedf["scentroid"]==row["scentroid"]) & (linedf["acentroid"]==row["acentroid"])]
    lines.append(sub)

olines = pd.concat(lines)

In [ ]:
# get sequences
seq_dict = {record.description.split(" ")[0]: str(record.seq) for record in SeqIO.parse(open(rootdir + "trees/all_rps3.faa"), "fasta")}

In [ ]:
sgolines = pd.concat([sglines, olines])
sgolines = sgolines.merge(lineagg[["scentroid", "acentroid", "env_broad", "stax", "atax", "type"]], how="left", on=["scentroid", "acentroid"])
sgolines["sseq"] = sgolines["scluster"].map(seq_dict)
sgolines["aseq"] = sgolines["acluster"].map(seq_dict)
sgolines = sgolines[["sample", "env_broad", "scluster", "stax", "sseq", "acluster", "atax", "aseq", "type"]]
sgolines.columns = ["run_id", "habitat_broad", "saccharibacateria_graftm_s3", "saccharibacteria_graftm_taxonomy", "saccharibacteria_graftm_seq", "actinobacteria_graftm_s3", "actinobacteria_graftm_taxonomy", "actinobacteria_graftm_seq", "co-occurrence type"]

In [ ]:
sgolines.sort_values(["habitat_broad", "co-occurrence type", "run_id"], ascending=[True,True, True]).to_csv(rootdir + "metadata/supp_table_4.csv", index=False)

# gracili/sr1 co-occurrence

In [ ]:
cmdir(rootdir + "marker_gene")

In [ ]:
# samples of interest
runs = covsub[covsub["taxcat"].isin(["Absconditabacteria", "Gracilibacteria"])]["run_id"].unique()
info = runsub[runsub["run_id"].isin(runs)].drop_duplicates(["run_id", "read_path", "curated_assembly_path"])

### compute coverage values

In [ ]:
mappings = []

for key, row in info.iterrows():
    
    temp = []
    table = cresults[row["run_id"]]
    
    # pull scafs
    filename = rootdir + "marker_gene/" + row["run_id"] + ".names.txt"
    with open(filename, "w") as names:
        for centroid in table["centroid"].unique():
            names.write("_".join(centroid.split("_")[:-3]) + "\n")
    outfile = filename.replace("names.txt", "markerscafs.fna")
    call = "pullseq -n " + filename + " -i " + row["curated_assembly_path"] + \
        " > " + outfile
    sp.call(call, shell=True)
    
    temp.append("bowtie2-build " + outfile + " " + outfile)
    temp.append("bowtie2 -p 48 -x " + outfile + " -1 " + \
        row["read_path"].replace(".fa.gz", ".1.fastq.gz") + " -2 " + \
        row["read_path"].replace(".fa.gz", ".2.fastq.gz") + \
        " | shrinksam | samtools view -S -b > " + \
        outfile.replace("fna", "bam"))
    temp.append("samtools sort --threads 48 " + \
        outfile.replace("fna", "bam") + " > " + outfile.replace("fna", "sorted.bam"))
    temp.append("samtools index -@ 48 " + outfile.replace("fna", "sorted.bam"))
    mappings.append(temp)

In [ ]:
wrapperize(mappings, 10, rootdir + "marker_gene/map", "list")

In [ ]:
# get non zero sorted bams
sorted_bams = [item for item in glob.glob(rootdir + "/marker_gene/*sorted.bam") if os.stat(item).st_size != 0]

for bam in sorted_bams:
    
    coverm = "coverm contig --min-read-percent-identity .99 --output-format sparse -b " + bam + " -m" + \
        " count mean covered_fraction length rpkm > " + rootdir + "/marker_gene/" + \
        os.path.basename(bam).replace("sorted.bam", "") + "coverage_table.tsv"
    sp.call(coverm, shell=True)

In [ ]:
marker_cov = pd.concat([pd.read_csv(table, sep="\t") for table in glob.glob(rootdir + "marker_gene/*coverage_table.tsv")])
marker_cov = marker_cov[marker_cov["Covered Fraction"]>0.10]
msub = marker_cov[["Sample", "Contig", "Mean"]]
msub["run_id"] = msub["Sample"].apply(lambda x: x.split(".")[0])
msub.head()

### merge with clustering data

In [ ]:
all_cresults = pd.concat(cresults.values())
all_cresults = all_cresults[["centroid", "taxstring"]]
all_cresults["Contig"] = all_cresults["centroid"].apply(lambda x: "_".join(x.split("_")[:-3]))
all_cresults["lineage"] = all_cresults["taxstring"].apply(lambda x: x.split(";")[4] if len(x.split(";")) > 4 else x.split(";")[-1])
all_cresults.head()

In [ ]:
# merge
merge = msub.merge(all_cresults[["Contig", "lineage"]], on="Contig", how="left")
mgb = merge.groupby(["run_id", "lineage"], as_index=False).aggregate({"Mean":"sum"})

# compute relative cov
mgb = mgb.merge(mgb.groupby("run_id", as_index=False).aggregate({"Mean":"sum"})
    .rename(columns={"Mean":"total"}), how="left", on="run_id")
mgb["rel_cov"] = mgb.apply(lambda x: x["Mean"]/float(x["total"]), axis=1)
mgb["scaled_rel_cov"] = mgb["rel_cov"].apply(lambda x: math.log10(x/float(min(mgb["rel_cov"]))))

In [ ]:
env_data = covsub[["run_id", "env_broad", "env_narrow"]].drop_duplicates()
tallyho = mgb

In [ ]:
sublineages = pd.DataFrame(tallyho["lineage"].value_counts()).query("lineage>7").index.to_list()
sublineages = [item for item in sublineages if item not in 
    [' o__BD1-5',' o__Saccharimonadales',' d__Bacteria', ' o__UBA9983', 'o__UBA1400', ' o__Moranbacterales', ' c__ABY1', ' o__UBA6257', ' o__UBA1369']]
sublineages = [ ' o__BD1-5' , ' o__Absconditabacterales', ' o__Saccharimonadales'] + sublineages
tallysub = tallyho[tallyho["lineage"].isin(sublineages)]
tallypiv = tallysub.pivot("run_id", "lineage", "scaled_rel_cov").fillna(0)

### fix things up

In [ ]:
presdict = {}

for key, row in covsub.iterrows():
    
    if row["run_id"] not in presdict:
        presdict[row["run_id"]] = set([row["taxcat"]])
    else:
        presdict[row["run_id"]].add(row["taxcat"])

In [ ]:
for column in [" o__Saccharimonadales", " o__BD1-5", " o__Absconditabacterales"]:
    
    if "Sacchari" in column:
        name = "Saccharibacteria"
    elif "BD1-5" in column:
        name = "Gracilibacteria"
    elif "Abscondita" in column:
        name = "Absconditabacteria"
    
    temp = []
    for key, row in tallypiv.iterrows():
        if name in presdict[key]:
            temp.append(max(tallyho["scaled_rel_cov"]))
        else: temp.append(0)
    
    tallypiv[name] = temp
    tallypiv = tallypiv.drop(column, axis=1)
    sublineages.remove(column)
    sublineages.insert(0, name)

In [ ]:
tallymerge = tallypiv.reset_index().merge(env_data, how="left", on="run_id")
tallymerge["dummy"] = 0
sublineages.insert(3, "dummy")

In [ ]:
tallymerge.query("env_narrow=='wastewater'").query("Absconditabacteria!=0")

### plot it

In [ ]:
total = len(tallymerge["env_broad"].unique())
count=1

for env in tallymerge["env_broad"].unique():
    
    data = tallymerge[tallymerge["env_broad"]==env].sort_values(["Gracilibacteria", "Absconditabacteria"], ascending=[True, False])
    data.index = data["env_narrow"]
    data = data.drop(["run_id", "env_broad", "env_narrow"], axis=1)
    fraction = len(data)/float(len(tallymerge))
    plt.figure(figsize=(9,12))
    g= sns.heatmap(data[sublineages], linewidths=1, vmax=max(tallysub.scaled_rel_cov), 
        cmap=sns.light_palette(env2color[env], as_cmap=True), cbar=False, square=True)
    plt.xlabel("")
    plt.ylabel("")
    
    if count != total:
        plt.tick_params(bottom=False)
        g.set(xticks=[])
    else: plt.xticks(rotation=60, horizontalalignment="right")
        
    plt.savefig(rootdir + "figures/" + env + "_heatmap.svg", format="svg")
    plt.show()
    count+=1

# reanalyze PH data

In [ ]:
cmdir(rootdir + "eel_river")
cmdir(rootdir + "eel_river/graftm/")

### graftm

In [ ]:
with open(rootdir + "eel_river/graftm_wrapper.sh", "w") as outfile:

    for assembly in glob.glob(datadir + "PH*/assembly.d/*idba*/*_min1000.fa"):

        runid = os.path.basename(assembly).split("_scaffold")[0]
        call = "graftM graft --threads 20 --forward " + \
            assembly + " --input_sequence_type nucleotide --graftm_package " + gpkg + \
            " --filter_minimum 0 --output_directory " + rootdir + "/eel_river/graftm/" + runid
        outfile.write(call + "\n")

In [ ]:
# read in results
results = {}

for graftdir in glob.glob(rootdir + "eel_river/graftm/*/*/"):
    
    name = graftdir.split("/")[-2].split("_scaffold")[0]
    # find files
    taxinfo = glob.glob(graftdir + "/*read_tax.tsv")[0]
    results[name] = {"tax": pd.read_csv(taxinfo, sep="\t", header=None)}
    # only proceed if results
    if os.stat(taxinfo).st_size != 0:
        full_fasta = glob.glob(graftdir + "*orf.fa")[0]
        results[name]["fasta"] = full_fasta

### clustering

In [ ]:
with open(rootdir + "eel_river/all_markers.faa", "w") as outfile:
    for sample in results.keys():
        for record in SeqIO.parse(open(results[sample]["fasta"]), "fasta"):
            outfile.write(">" + record.description.split(" ")[0] + "\n" + str(record.seq) + "\n")

In [ ]:
call = "usearch -cluster_fast " + rootdir + "/eel_river/all_markers.faa -sort length -id 0.97 " + \
    "-centroids " + rootdir + "/eel_river/all_markers.centroids" + \
    " -uc " + rootdir + "/eel_river/all_markers.clusters"
sp.call(call, shell=True)

In [ ]:
eelclust = parse_usearch_clustering(rootdir + "eel_river/all_markers.clusters")
repscafs = ["_".join(gene.split("_")[:-3]) for gene in eelclust["centroid"].unique()]
len(repscafs)

In [ ]:
with open(rootdir + "eel_river/eel_markerscafs.fna", "w") as outfile:
    for assembly in glob.glob(datadir + "PH*/assembly.d/*idba*/*_min1000.fa"):
        for record in SeqIO.parse(open(assembly), "fasta"):
            if record.description.split(" ")[0] in repscafs:
                outfile.write(">" + record.description.split(" ")[0] + "\n" + str(record.seq) + "\n")

### mapping

In [ ]:
cmdir(rootdir + "eel_river/mapping")

In [ ]:
mapwrap = open(rootdir + "eel_river/mapwrap.sh", "w")
mscafs = rootdir + "eel_river/eel_markerscafs.fna"
mapwrap.write("bowtie2-build " + mscafs + " " + mscafs + "\n")

for read in glob.glob(datadir + "PH*/raw.d/*trim_clean.PE.1.fastq.gz"):
    
    name = os.path.basename(read).split("_trim")[0]
    out = rootdir + "eel_river/mapping/" + name + ".bam"
    mapwrap.write("bowtie2 -p 20 -x " + mscafs + " -1 " + \
        read + " -2 " + read.replace(".1.", ".2.") + \
        " | shrinksam | samtools view -S -b > " + \
        out + "\n")
    mapwrap.write("amtools sort --threads 20 " + \
        out + " > " + out.replace("bam", "sorted.bam") + "\n")
    mapwrap.write("samtools index -@ 18 " + out.replace("bam", "sorted.bam") + "\n")
    
mapwrap.close()

In [ ]:
# extract coverage information
coverm = "coverm contig --min-read-percent-identity .97 --output-format sparse -b " + rootdir + \
    "eel_river/mapping/*sorted.bam -m count mean covered_fraction length rpkm > " + rootdir + "eel_river/coverage_table.csv"
print(coverm)

In [ ]:
marker_cov = pd.read_csv(rootdir + "eel_river/coverage_table.csv", sep="\t")
marker_cov = marker_cov[marker_cov["Covered Fraction"]>0.10]
msub = marker_cov[["Sample", "Contig", "Mean"]]
msub.head()

### quick corr

In [ ]:
all_eel = pd.concat(item["tax"] for key, item in results.items())
all_eel.columns = ["gene", "tax"]
all_eel["Contig"] = all_eel["gene"].apply(lambda x: "_".join(x.split("_")[:-3]))
mmerge = msub.merge(all_eel[["Contig","tax"]], how="left")

In [ ]:
from scipy.stats import pearsonr
corrs = defaultdict(list)

piv = mmerge.pivot("Sample", "Contig", "Mean").fillna(0)

for col in piv.columns:
    if col != "PH2015_14_scaffold_705":
        r, pval = pearsonr(piv["PH2015_14_scaffold_705"], piv[col])
        corrs["row"].append("PH2015_14_scaffold_705")
        corrs["col"].append(col)
        corrs["r"].append(r)
        corrs["pval"].append(pval)
        
corrdf = pd.DataFrame(corrs)

In [ ]:
subdfs = {}

for i,col in enumerate(corrdf.sort_values("r", ascending=False)["col"][0:9]):
    subtable = piv[["PH2015_14_scaffold_705", col]].reset_index()
    subtable["taxon"] = col
    subtable.columns = ["sample", "sr1_mean_cov", "taxon_mean_cov", "taxon"]
    subdfs[i] = subtable

newdata = pd.concat(subdfs.values())

In [ ]:
top10 = corrdf.sort_values("r", ascending=False).merge(mmerge[["Contig", "tax"]], left_on="col", right_on="Contig", how="left").drop_duplicates().head(25)

In [ ]:
supp = top10[["col", "tax", "r", "pval"]]
supp.columns = ["representative_scaffold", "graftm_taxonomy", "r", "pval"]
supp.to_csv(rootdir + "metadata/eel_supp_table.csv", index=False)